# Creating embeddings for the LLM
1. Tokenization
2. Data sampling
3. Token embeddings

## Tokenization
- BPE (Byte pair enconding) tokenizer was used to train LLMs such as GPT-2, GPT-3 and the original model used in ChatGPT

In [1]:
from importlib.metadata import version
import tiktoken # An OpenAI library which impelements BPE efficiently
print(f"titoken version: {version('tiktoken')}")

titoken version: 0.9.0


In [2]:
# Instantiate BPE tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
# Encode a text
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownplace"
)

integers = tokenizer.encode(text, allowed_special= {"<|endoftext|>"})
print(f"Encoded text: {integers}")

Encoded text: [15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 5372]


In [4]:
# Decode back to text
decoded_text = tokenizer.decode(integers)
print(f"Decoded text: {decoded_text}")

Decoded text: Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownplace


In [5]:
# Exercise: Encode and decode a text
text = "Akwirw ier"
integers = tokenizer.encode(text)
decoded_text = tokenizer.decode(integers)
print(f"Encoded text: {integers}")
print(f"Decoded text: {decoded_text}")

Encoded text: [33901, 86, 343, 86, 220, 959]
Decoded text: Akwirw ier


## Data sampling with a sliding window

In [6]:
# Dowload a text corpus
import urllib.request
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
filepath = "data/the-verdict.txt"
urllib.request.urlretrieve(url, filepath)

('data/the-verdict.txt', <http.client.HTTPMessage at 0x105028f50>)

In [7]:
with open(filepath, "r", encoding= "utf-8") as f:
    raw_text = f.read()
print(f"Text length: {len(raw_text)} characters")
print(f"Text sample: {raw_text[:99]}")

Text length: 20479 characters
Text sample: I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [8]:
enc_text = tokenizer.encode(raw_text)
print(f"Encoded text length: {len(enc_text)} tokens")

Encoded text length: 5145 tokens


In [9]:
# Remove the first 50 tokens from the encoded text
enc_sample = enc_text[50:]

In [10]:
# Creating input-target pairs for the next word prediction task
# x contains the input tokens, and y contains the target tokens, which are the inputs shifted by one position

context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [11]:
# construct input-target encode pairs

for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [12]:
# contruct input-target word pairs
for i in range(1, context_size + 1):
    context = tokenizer.decode(enc_sample[:i])
    desired = tokenizer.decode([enc_sample[i]]) # Array containing single token
    print(context, "---->", desired)

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [13]:
# Creating inputs and targets as tensors
# Pytorch has built-in Dataset and DataLoader classes for efficient data loading

import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokkenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokkenizer.encode(text)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]




/Users/zamber/Learning/build_llm_from_scratch/build_llm_from_scratch/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [14]:
# A data loader to generate batches with input pairs

def create_dataloader_v1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [15]:
# create a data loader with a batch size of 1 and context window 4
with open(filepath, "r", encoding="utf-8") as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
# Iterate through the data loader
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [16]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [17]:
# data loader with max_length 2 and stride 2
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=2, stride=2, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(f"First batch: {first_batch}")
second_batch = next(data_iter)
print(f"Second batch: {second_batch}")

First batch: [tensor([[ 40, 367]]), tensor([[ 367, 2885]])]
Second batch: [tensor([[2885, 1464]]), tensor([[1464, 1807]])]


In [18]:
# data loader with max_length 8 and stride 2
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=8, stride=2, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(f"First batch: {first_batch}")
second_batch = next(data_iter)
print(f"Second batch: {second_batch}")

First batch: [tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]
Second batch: [tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]]), tensor([[ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])]


In [19]:
# data loader with batch_size 8, max_length 4 and stride 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f"Inputs:\n {inputs}")
print(f"Targets:\n {targets}")

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


Note: Equal max_length (Window size) and stride avoids overlap between the batches. Overlaps could lead to increased overfitting.

## Creating token embeddings

In [1]:
import torch
torch.__version__

/Users/zamber/Learning/build_llm_from_scratch/build_llm_from_scratch/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


'2.2.2'

In [21]:
torch.cuda.is_available()  # Check if CUDA is available

False

In [22]:
torch.backends.mps.is_available()  # Check if MPS (Metal Performance Shaders) is available on macOS

True

In [23]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3

# Create a random embedding layer
torch.manual_seed(42)  # For reproducibility
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 1.9269,  1.4873, -0.4974],
        [ 0.4396, -0.7581,  1.0783],
        [ 0.8008,  1.6806,  0.3559],
        [-0.6866,  0.6105,  1.3347],
        [-0.2316,  0.0418, -0.2516],
        [ 0.8599, -0.3097, -0.3957]], requires_grad=True)


In [24]:
# Get embedding vector for a token
print(f"Embedding for token {input_ids[1]}: {embedding_layer(input_ids[1])}")

Embedding for token 3: tensor([-0.6866,  0.6105,  1.3347], grad_fn=<EmbeddingBackward0>)


In [25]:
# Get the embeddings for the input_ids
embeddings = embedding_layer(input_ids)
print(f"Embeddings for input_ids: \n{embeddings}")

Embeddings for input_ids: 
tensor([[ 0.8008,  1.6806,  0.3559],
        [-0.6866,  0.6105,  1.3347],
        [ 0.8599, -0.3097, -0.3957],
        [ 0.4396, -0.7581,  1.0783]], grad_fn=<EmbeddingBackward0>)


### Positional embeddings

In [26]:
vocab_size = 50257
output_dim = 256 # This is for experimentation. in GPT-3, embedding dimension is 12288
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)


In [27]:
# Creata a data loader with batch_size 8, max_length 4 and stride 4
batch_size = 8
max_length = 4
stride = 4

dataloader = create_dataloader_v1(raw_text, batch_size=batch_size, max_length=max_length, stride=stride, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f"Token IDs:\n {inputs}")
print(f"\n Inputs shape:\n {inputs.shape}")

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

 Inputs shape:
 torch.Size([8, 4])


In [28]:
# Use the token embedding layer to embed input tokens into 256-dimensional vectors
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [29]:
torch.arange(max_length)

tensor([0, 1, 2, 3])

In [30]:
# positional embedding layer with absulute embedding approach like OpenAI's GPT models
context_length = max_length # same window size as the token embedding layer
positional_embedding_layer = torch.nn.Embedding(context_length, output_dim)
# Get the positional embeddings for the input tokens
pos_embeddings = positional_embedding_layer(torch.arange(context_length))
pos_embeddings.shape

torch.Size([4, 256])

In [31]:
# Add the positional embeddings to the token embeddings
input_embeddings = token_embeddings + pos_embeddings
input_embeddings.shape

torch.Size([8, 4, 256])